In [ ]:
!pip install torch torchvision torchaudio

In [ ]:
!pip install cudatoolkit=11.7

In [ ]:
!pip install mamba-ssm notebook

In [ ]:
!pip install numpy matplotlib

In [ ]:
import os
import numpy as np
import torch
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms
from mamba_ssm import Mamba
import torchvision

In [1]:
class ImageDataset(Dataset):
    def __init__(self, directory, transform=None):
        self.directory = directory
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.class_names = os.listdir(directory)
        for label, class_name in enumerate(self.class_names):
            class_folder = os.path.join(directory, class_name)
            if os.path.isdir(class_folder):
                for image_name in os.listdir(class_folder):
                    self.image_paths.append(os.path.join(class_folder, image_name))
                    self.labels.append(label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        label = self.labels[idx]
        image = np.array(plt.imread(image_path))
        if self.transform:
            image = self.transform(image)
        return image, label

NameError: name 'Dataset' is not defined

In [ ]:
img_height, img_width = 256, 256
batch_size = 32
epochs = 20
num_classes = 116
learning_rate = 0.001
d_model = 16  # Model dimension d_model (from mamba_ssm model)

In [ ]:
# Model setup using Mamba from mamba_ssm
model = Mamba(
    d_model=d_model,
    d_state=16,   # SSM state expansion factor
    d_conv=4,     # Local convolution width (not used in CNN)
    expand=2,     # Block expansion factor
).to("cuda")

In [ ]:
# Optimizer and Loss function
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
train_dataset = ImageDataset('train', transform=torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Resize((img_height, img_width)),
]))
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = ImageDataset('test', transform=torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Resize((img_height, img_width)),
]))
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct_preds = 0
    total_preds = 0
    for images, labels in train_loader:
        images, labels = images.to("cuda"), labels.to("cuda")
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct_preds += (predicted == labels).sum().item()
        total_preds += labels.size(0)

    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = correct_preds / total_preds
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}")

In [ ]:
model.eval()
correct_preds = 0
total_preds = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to("cuda"), labels.to("cuda")
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        correct_preds += (predicted == labels).sum().item()
        total_preds += labels.size(0)

test_accuracy = correct_preds / total_preds
print(f"Test Accuracy: {test_accuracy:.4f}")